In [1]:
from pathlib import Path
import json
from functools import reduce
import math
import datetime as dt
import pytz 
from itertools import product
from collections import OrderedDict
import time
import sys

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.ops as so

%load_ext autoreload
%autoreload 2

sys.path.append('../')
import affordability as af


# Prepare rent data

In [ ]:
# Reshape and merge all rent data sets

def clean(f, name):
    f = f.copy()
    f = f.rename(columns={
        'SAU': 'au2001',
        'Property_Type': 'property_type',
        'Bedrooms': '#bedrooms'
    })

    # Drop subtotals
    cond = False
    for col in ['au2001', 'property_type', '#bedrooms']:
        cond |= f[col].str.contains('total', case=False)

    f = f[~cond].copy()
    
    # Reshape
    id_vars = ['au2001', 'property_type', '#bedrooms']
    value_vars = [c for c in f.columns if '-' in c]
    f = pd.melt(f, id_vars=id_vars, value_vars=value_vars,
      var_name='quarter', value_name=name)
    
    return f

paths = [
    af.DATA_DIR/'raw'/'Detailed Bonds Lodged.csv',
    af.DATA_DIR/'raw'/'Detailed Mean Rents.csv',
    af.DATA_DIR/'raw'/'Detailed Geomean Rents.csv',
    af.DATA_DIR/'raw'/'Detailed Synthetic Lower Quartile Rents.csv',
    af.DATA_DIR/'raw'/'Detailed Synthetic Upper Quartile Rents.csv',
]
names = ['rent_count', 'rent_mean', 'rent_geo_mean', 'rent_synthetic_lower_quartile', 'rent_synthetic_upper_quartile']
frames = []
for path, name in zip(paths, names):
    f = pd.read_csv(path, dtype={'SAU': str})
    frames.append(clean(f, name))
    
f = reduce(lambda x, y: pd.merge(x, y), frames)

# Merge in region data
path = af.DATA_DIR/'au2001.csv'
g = pd.read_csv(path, dtype={'au2001': str})
f = f.merge(g)

# Write to file
path = af.DATA_DIR/'rents.csv'
f.to_csv(str(path), index=False)
f[f['rent_count'].notnull()].head()

# Explore rents

In [105]:
path = af.DATA_DIR/'rents.csv'
rents = pd.read_csv(path, dtype={'au2001': str})
rents.head()


,au2001,property_type,#bedrooms,quarter,rent_count,rent_mean,rent_geo_mean,rent_synthetic_lower_quartile,rent_synthetic_upper_quartile,au_name,territory,region,rental_area
0,500100,House,2,1993-03-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
1,500100,House,3,1993-03-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
2,500100,House,2,1993-06-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
3,500100,House,3,1993-06-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North
4,500100,House,2,1993-09-01,NaN,NaN,NaN,NaN,NaN,Awanui,Far North District,Northland,Rural Far North


In [106]:
# Slice in time and aggregate 

def hits(group):
    d = {}
    d['hit_frac'] = group['rent_count'].dropna().shape[0]/group['rent_count'].shape[0]
    return pd.Series(d)

agg_rents = af.aggregate_rents(rents, '2016-12-01')
agg_rents.head()

,rental_area,#bedrooms,region,rent_count,rent_geo_mean,rent_mean,territory
0,Addington,1,Canterbury,133.0,214.863943,218.654135,Christchurch City
1,Addington,2,Canterbury,56.0,339.402991,342.535714,Christchurch City
2,Addington,3,Canterbury,54.0,414.203127,418.796296,Christchurch City
3,Addington,4,Canterbury,5.0,488.000000,494.000000,Christchurch City
4,Addington,5+,Canterbury,NaN,NaN,NaN,Christchurch City


In [111]:
# What fraction of rental data do we have by #bedrooms?

date = '2016-09-01'
f = af.aggregate_rents(rents, date, groupby_cols=('au2001', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('census area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())

f = af.aggregate_rents(rents, date, groupby_cols=('rental_area', '#bedrooms'))
cond = f['region'] == 'Auckland'
print('rental area units')
print(f[cond].copy().groupby('#bedrooms').apply(hits).reset_index())


census area units
  #bedrooms  hit_frac
0         1  0.396226
1         2  0.662791
2         3  0.843750
3         4  0.602305
4        5+  0.115016
rental area units
  #bedrooms  hit_frac
0         1  0.767677
1         2  0.959596
2         3  1.000000
3         4  0.919192
4        5+  0.252525


# Prepare regional slices of data

In [4]:

path = af.DATA_DIR/'rents.csv'
rents = pd.read_csv(path, dtype={'au2001': str})

for region in af.REGIONS:
    root = af.DATA_DIR/region
    if not root.exists():
        root.mkdir()
        
    region_c = region.capitalize()
       
    # Rents slice
    f = rents[rents['region'] == region_c].copy()
    path = root/'rents.csv'
    f.to_csv(str(path), index=False)
    